# Análise de Dados - Óticas Taty Mello

Este notebook realiza análise exploratória dos dados de ordens de serviço das óticas.

## Objetivos:
- Carregar e analisar planilhas OS_NOVA
- Identificar padrões nos dados
- Detectar duplicatas de clientes
- Analisar distribuição de dioptrias
- Gerar relatórios e visualizações

In [ ]:
# Importações necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configurações
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Bibliotecas carregadas com sucesso!")

In [ ]:
# Importar nossas classes personalizadas
import sys
sys.path.append('../scripts')
sys.path.append('../app/services')

from analisar_os import AnalisadorOS
from deduplicacao import DeduplicadorClientes

print("✅ Classes personalizadas carregadas!")

## 1. Carregamento e Análise Inicial dos Dados

In [ ]:
# Inicializar analisador
analisador = AnalisadorOS(diretorio_dados="../data/raw")

# Listar arquivos disponíveis
arquivos_raw = list(Path("../data/raw").glob("*.xlsx"))
print(f"Arquivos encontrados: {len(arquivos_raw)}")
for arquivo in arquivos_raw:
    print(f"  📁 {arquivo.name}")

In [ ]:
# Se houver arquivos, processar o primeiro como exemplo
if arquivos_raw:
    arquivo_exemplo = arquivos_raw[0]
    print(f"Analisando arquivo: {arquivo_exemplo.name}")
    
    # Carregar dados
    df_original = analisador.carregar_planilha(arquivo_exemplo)
    
    print(f"\n📊 Informações básicas:")
    print(f"  - Linhas: {len(df_original)}")
    print(f"  - Colunas: {len(df_original.columns)}")
    print(f"  - Tamanho em memória: {df_original.memory_usage(deep=True).sum() / 1024:.1f} KB")
    
    # Mostrar colunas
    print(f"\n📋 Colunas encontradas:")
    for i, col in enumerate(df_original.columns):
        print(f"  {i+1:2d}. {col}")
        
    # Primeiras linhas
    print(f"\n🔍 Primeiras 3 linhas:")
    display(df_original.head(3))
else:
    print("❌ Nenhum arquivo encontrado em data/raw/")
    print("   Copie alguns arquivos OS_NOVA*.xlsx para data/raw/ e execute novamente")

## 2. Padronização e Limpeza dos Dados

In [ ]:
if arquivos_raw:
    # Padronizar colunas
    df_limpo = analisador.padronizar_colunas(df_original.copy())
    
    print("🔧 Colunas após padronização:")
    for i, col in enumerate(df_limpo.columns):
        print(f"  {i+1:2d}. {col}")
    
    # Mostrar comparação antes/depois
    print("\n📋 Mapeamento de colunas:")
    for orig, novo in zip(df_original.columns, df_limpo.columns):
        if orig != novo:
            print(f"  '{orig}' → '{novo}'")

In [ ]:
if arquivos_raw:
    # Análise de dados faltantes
    print("🔍 Análise de dados faltantes:")
    missing = df_limpo.isnull().sum()
    missing_pct = (missing / len(df_limpo)) * 100
    
    missing_df = pd.DataFrame({
        'Coluna': missing.index,
        'Valores_Faltantes': missing.values,
        'Percentual': missing_pct.values
    }).sort_values('Percentual', ascending=False)
    
    display(missing_df.head(10))
    
    # Visualizar dados faltantes
    plt.figure(figsize=(12, 6))
    missing_df_plot = missing_df[missing_df['Percentual'] > 0]
    
    plt.subplot(1, 2, 1)
    plt.barh(missing_df_plot['Coluna'][:10], missing_df_plot['Percentual'][:10])
    plt.xlabel('Percentual de Valores Faltantes')
    plt.title('Top 10 Colunas com Dados Faltantes')
    plt.gca().invert_yaxis()
    
    plt.subplot(1, 2, 2)
    plt.hist(missing_df_plot['Percentual'], bins=20, alpha=0.7)
    plt.xlabel('Percentual de Valores Faltantes')
    plt.ylabel('Número de Colunas')
    plt.title('Distribuição de Dados Faltantes')
    
    plt.tight_layout()
    plt.show()

## 3. Análise de Clientes e Detecção de Duplicatas

In [ ]:
if arquivos_raw and 'nome' in df_limpo.columns:
    # Análise básica de clientes
    print("👥 Análise de Clientes:")
    print(f"  - Total de registros: {len(df_limpo)}")
    print(f"  - Clientes únicos (por nome): {df_limpo['nome'].nunique()}")
    print(f"  - Registros com nome vazio: {df_limpo['nome'].isnull().sum()}")
    
    if 'cpf' in df_limpo.columns:
        print(f"  - CPFs únicos: {df_limpo['cpf'].nunique()}")
        print(f"  - Registros com CPF vazio: {df_limpo['cpf'].isnull().sum()}")
    
    if 'telefone' in df_limpo.columns:
        print(f"  - Telefones únicos: {df_limpo['telefone'].nunique()}")
        print(f"  - Registros com telefone vazio: {df_limpo['telefone'].isnull().sum()}")
    
    # Top 10 clientes por número de OS
    top_clientes = df_limpo['nome'].value_counts().head(10)
    print(f"\n🏆 Top 10 clientes com mais OS:")
    for nome, count in top_clientes.items():
        print(f"  {count:2d} OS - {nome}")

In [ ]:
if arquivos_raw and 'nome' in df_limpo.columns:
    # Detectar duplicatas usando nossa classe
    print("🔍 Detectando duplicatas...")
    
    deduplicador = DeduplicadorClientes()
    
    # Usar apenas uma amostra se o dataset for muito grande
    df_amostra = df_limpo.head(100) if len(df_limpo) > 100 else df_limpo
    
    duplicatas = deduplicador.encontrar_duplicatas(df_amostra)
    
    print(f"✅ Encontradas {len(duplicatas)} possíveis duplicatas")
    
    if duplicatas:
        # Gerar relatório
        relatorio_duplicatas = deduplicador.gerar_relatorio_duplicatas(duplicatas)
        
        print("\n📊 Top 5 duplicatas com maior score:")
        display(relatorio_duplicatas.head())
        
        # Estatísticas das duplicatas
        print("\n📈 Estatísticas das duplicatas:")
        print(f"  - Duplicatas com alta confiança: {(relatorio_duplicatas['Confianca'] == 'alta').sum()}")
        print(f"  - Duplicatas com média confiança: {(relatorio_duplicatas['Confianca'] == 'media').sum()}")
        print(f"  - Duplicatas com baixa confiança: {(relatorio_duplicatas['Confianca'] == 'baixa').sum()}")
        
        print(f"\n  - Recomendação 'merge': {(relatorio_duplicatas['Recomendacao'] == 'merge').sum()}")
        print(f"  - Recomendação 'revisar': {(relatorio_duplicatas['Recomendacao'] == 'revisar').sum()}")
        print(f"  - Recomendação 'ignorar': {(relatorio_duplicatas['Recomendacao'] == 'ignorar').sum()}")

## 4. Análise de Dioptrias

In [ ]:
if arquivos_raw:
    # Identificar colunas de dioptrias
    colunas_dioptria = [col for col in df_limpo.columns if any(x in col.lower() for x in ['od_', 'oe_', 'esferico', 'cilindrico', 'eixo', 'adicao'])]
    
    print(f"🔍 Colunas de dioptria encontradas: {len(colunas_dioptria)}")
    for col in colunas_dioptria:
        print(f"  - {col}")
    
    if colunas_dioptria:
        # Análise básica das dioptrias
        print("\n📊 Estatísticas das dioptrias:")
        
        for col in colunas_dioptria[:6]:  # Limitar a 6 colunas
            if df_limpo[col].dtype in ['float64', 'int64']:
                valores_validos = df_limpo[col].dropna()
                if len(valores_validos) > 0:
                    print(f"\n  {col}:")
                    print(f"    - Valores válidos: {len(valores_validos)}")
                    print(f"    - Média: {valores_validos.mean():.2f}")
                    print(f"    - Mediana: {valores_validos.median():.2f}")
                    print(f"    - Min/Max: {valores_validos.min():.2f} / {valores_validos.max():.2f}")
    else:
        print("❌ Nenhuma coluna de dioptria identificada")

In [ ]:
if arquivos_raw and colunas_dioptria:
    # Visualizações das dioptrias
    print("📈 Gerando visualizações das dioptrias...")
    
    # Selecionar colunas numéricas de dioptria
    colunas_numericas = [col for col in colunas_dioptria if df_limpo[col].dtype in ['float64', 'int64']]
    
    if len(colunas_numericas) >= 2:
        fig, axes = plt.subplots(2, min(3, len(colunas_numericas)), figsize=(15, 10))
        axes = axes.flatten() if len(colunas_numericas) > 3 else [axes] if len(colunas_numericas) == 1 else axes
        
        for i, col in enumerate(colunas_numericas[:6]):
            ax = axes[i] if i < len(axes) else None
            if ax is not None:
                valores = df_limpo[col].dropna()
                if len(valores) > 0:
                    if i < 3:
                        # Histogramas na primeira linha
                        ax.hist(valores, bins=30, alpha=0.7, edgecolor='black')
                        ax.set_title(f'Distribuição - {col}')
                        ax.set_xlabel('Valor')
                        ax.set_ylabel('Frequência')
                    else:
                        # Boxplots na segunda linha
                        ax.boxplot(valores)
                        ax.set_title(f'Boxplot - {col}')
                        ax.set_ylabel('Valor')
        
        # Remover subplots vazios
        for i in range(len(colunas_numericas), len(axes)):
            fig.delaxes(axes[i])
        
        plt.tight_layout()
        plt.show()
    
    else:
        print("❌ Não há colunas numéricas suficientes para visualização")

## 5. Análise Temporal (se disponível)

In [ ]:
if arquivos_raw:
    # Identificar colunas de data
    colunas_data = [col for col in df_limpo.columns if any(x in col.lower() for x in ['data', 'date'])]
    
    print(f"📅 Colunas de data encontradas: {len(colunas_data)}")
    for col in colunas_data:
        print(f"  - {col}")
    
    if colunas_data:
        # Tentar converter para datetime
        for col in colunas_data[:2]:  # Limitar a 2 colunas
            try:
                df_limpo[f'{col}_dt'] = pd.to_datetime(df_limpo[col], errors='coerce')
                valores_validos = df_limpo[f'{col}_dt'].dropna()
                
                if len(valores_validos) > 0:
                    print(f"\n📊 Análise temporal - {col}:")
                    print(f"  - Datas válidas: {len(valores_validos)}")
                    print(f"  - Período: {valores_validos.min().date()} até {valores_validos.max().date()}")
                    print(f"  - Amplitude: {(valores_validos.max() - valores_validos.min()).days} dias")
                    
                    # Distribuição por mês
                    if len(valores_validos) > 10:
                        plt.figure(figsize=(12, 4))
                        
                        plt.subplot(1, 2, 1)
                        valores_validos.dt.to_period('M').value_counts().sort_index().plot(kind='bar')
                        plt.title(f'Distribuição Mensal - {col}')
                        plt.xlabel('Mês')
                        plt.ylabel('Quantidade')
                        plt.xticks(rotation=45)
                        
                        plt.subplot(1, 2, 2)
                        valores_validos.dt.dayofweek.value_counts().sort_index().plot(kind='bar')
                        plt.title(f'Distribuição por Dia da Semana - {col}')
                        plt.xlabel('Dia da Semana (0=Segunda)')
                        plt.ylabel('Quantidade')
                        
                        plt.tight_layout()
                        plt.show()
                        
            except Exception as e:
                print(f"❌ Erro ao processar coluna {col}: {e}")
    else:
        print("❌ Nenhuma coluna de data identificada")

## 6. Relatório Final e Recomendações

In [ ]:
if arquivos_raw:
    print("📋 RELATÓRIO FINAL")
    print("=" * 50)
    
    print(f"\n📊 RESUMO DOS DADOS:")
    print(f"  - Arquivo analisado: {arquivo_exemplo.name}")
    print(f"  - Total de registros: {len(df_original):,}")
    print(f"  - Total de colunas: {len(df_original.columns)}")
    print(f"  - Registros após limpeza: {len(df_limpo):,}")
    
    if 'nome' in df_limpo.columns:
        print(f"\n👥 CLIENTES:")
        print(f"  - Clientes únicos: {df_limpo['nome'].nunique():,}")
        print(f"  - Taxa de duplicação: {((len(df_limpo) - df_limpo['nome'].nunique()) / len(df_limpo) * 100):.1f}%")
        
        if 'duplicatas' in locals() and duplicatas:
            print(f"  - Duplicatas detectadas: {len(duplicatas)}")
            merge_recomendado = sum(1 for d in duplicatas if d.recomendacao == 'merge')
            print(f"  - Merge recomendado: {merge_recomendado}")
    
    if colunas_dioptria:
        print(f"\n👓 DIOPTRIAS:")
        print(f"  - Colunas de dioptria: {len(colunas_dioptria)}")
        
        # Calcular completude dos dados de dioptria
        for col in colunas_dioptria[:4]:
            if col in df_limpo.columns:
                completude = (df_limpo[col].notna().sum() / len(df_limpo)) * 100
                print(f"  - {col}: {completude:.1f}% preenchido")
    
    print(f"\n🔧 QUALIDADE DOS DADOS:")
    total_missing = df_limpo.isnull().sum().sum()
    total_cells = len(df_limpo) * len(df_limpo.columns)
    completude_geral = ((total_cells - total_missing) / total_cells) * 100
    print(f"  - Completude geral: {completude_geral:.1f}%")
    print(f"  - Células vazias: {total_missing:,}")
    
    print(f"\n💡 RECOMENDAÇÕES:")
    print(f"  1. Implementar processo de deduplicação automática")
    print(f"  2. Padronizar formato dos campos (CPF, telefone, etc.)")
    print(f"  3. Criar validações para entrada de dados")
    print(f"  4. Estabelecer campos obrigatórios")
    
    if len(colunas_dioptria) > 0:
        print(f"  5. Implementar validações específicas para dioptrias")
    
    if len(colunas_data) > 0:
        print(f"  6. Padronizar formato de datas")
    
    print(f"\n✅ Análise concluída!")
else:
    print("❌ Não foi possível realizar a análise.")
    print("   Certifique-se de copiar arquivos para data/raw/ e executar novamente.")

## 7. Exportar Dados Limpos

In [ ]:
if arquivos_raw and 'df_limpo' in locals():
    # Exportar dados limpos
    output_dir = Path("../data/processed")
    output_dir.mkdir(exist_ok=True)
    
    output_file = output_dir / f"dados_limpos_{arquivo_exemplo.stem}.xlsx"
    
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        # Dados limpos
        df_limpo.to_excel(writer, sheet_name='dados_limpos', index=False)
        
        # Relatório de duplicatas (se existir)
        if 'relatorio_duplicatas' in locals() and not relatorio_duplicatas.empty:
            relatorio_duplicatas.to_excel(writer, sheet_name='duplicatas', index=False)
        
        # Estatísticas
        stats_df = pd.DataFrame({
            'Metrica': [
                'Total de registros originais',
                'Total de registros limpos',
                'Total de colunas',
                'Clientes únicos',
                'Duplicatas detectadas',
                'Completude geral (%)'
            ],
            'Valor': [
                len(df_original),
                len(df_limpo),
                len(df_limpo.columns),
                df_limpo['nome'].nunique() if 'nome' in df_limpo.columns else 'N/A',
                len(duplicatas) if 'duplicatas' in locals() else 0,
                f"{completude_geral:.1f}%" if 'completude_geral' in locals() else 'N/A'
            ]
        })
        
        stats_df.to_excel(writer, sheet_name='estatisticas', index=False)
    
    print(f"✅ Dados exportados para: {output_file}")
    print(f"   Sheets criados:")
    print(f"   - dados_limpos: {len(df_limpo)} registros")
    
    if 'relatorio_duplicatas' in locals() and not relatorio_duplicatas.empty:
        print(f"   - duplicatas: {len(relatorio_duplicatas)} registros")
    
    print(f"   - estatisticas: resumo da análise")
else:
    print("❌ Não há dados para exportar")